In [1]:
# screener_dma2green_exits.py
"""
Exit checker for positions generated by screener_dma2green_entries.py

Reads portfolio/open_positions.csv and checks exits on the latest bar:
  Priority: HSL -> TSL -> Time (>= max_hold_days) -> TwoRedBelow20 -> Protective

Sends Telegram alerts for any positions that meet exit conditions.

ENV required for Telegram:
  TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID

Run:
  python screener_dma2green_exits.py

Option:
  AUTO_REMOVE_ON_ALERT = True to remove exited positions automatically.
"""

import os
import time
import pathlib
import traceback
from datetime import datetime, timedelta
from typing import Dict, List

import pandas as pd
import numpy as np
import yfinance as yf
import requests

# ----------- USER CONFIG -----------
AUTO_REMOVE_ON_ALERT = False  # set True to auto-remove positions after alert
START_DATE = "2015-01-01"
END_DATE = None  # None -> today
DMA_FAST = 10
DMA_SLOW = 20

# Telegram
ENABLE_TELEGRAM = True
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "")
TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID", "")

# Paths
CACHE_DIR = pathlib.Path("cache")
PORTFOLIO_DIR = pathlib.Path("portfolio")
OPEN_POSITIONS_CSV = PORTFOLIO_DIR / "open_positions.csv"
EXIT_ALERTS_DIR = pathlib.Path("alerts"); EXIT_ALERTS_DIR.mkdir(parents=True, exist_ok=True)

CACHE_DIR.mkdir(parents=True, exist_ok=True)
PORTFOLIO_DIR.mkdir(parents=True, exist_ok=True)

# ----------- Helpers (same as entry script) -----------
def normalize_df_columns(df: pd.DataFrame) -> pd.DataFrame:
    if hasattr(df, "columns") and getattr(df.columns, "nlevels", 1) > 1:
        df.columns = ["_".join([str(c) for c in col if c is not None]).strip() for c in df.columns.values]
    cols = list(df.columns)
    mapping = {}
    lower_map = {c.lower(): c for c in cols}
    for name in ['close','high','low','open','volume']:
        if name in lower_map:
            mapping[lower_map[name]] = name.capitalize()
        else:
            match = next((c for c in cols if name in c.lower()), None)
            if match:
                mapping[match] = name.capitalize()
    if mapping:
        df = df.rename(columns=mapping)
    return df

def collapse_duplicate_columns_take_first(df: pd.DataFrame) -> pd.DataFrame:
    if df.columns.duplicated().any():
        df = df.groupby(df.columns, axis=1).first()
    return df

def _cache_path_for_ticker(ticker: str) -> pathlib.Path:
    safe = ticker.replace('/', '_').replace(':', '_')
    return CACHE_DIR / f"{safe}.csv"

def _read_cache(ticker: str) -> pd.DataFrame:
    p = _cache_path_for_ticker(ticker)
    if p.exists() and p.stat().st_size > 0:
        try:
            return pd.read_csv(p, index_col=0, parse_dates=True)
        except Exception:
            try: p.unlink()
            except Exception: pass
    return None

def _write_cache(ticker: str, df: pd.DataFrame) -> None:
    p = _cache_path_for_ticker(ticker)
    try:
        df.to_csv(p)
    except Exception:
        pass

def download_with_retry(ticker: str, start: str, end: str, interval="1d", max_retries=3, backoff_sec=2):
    attempt = 0
    while attempt < max_retries:
        try:
            df = yf.download(ticker, start=start, interval=interval, end=end,
                             auto_adjust=True, progress=False, threads=True, multi_level_index=False)
            return df
        except Exception as e:
            attempt += 1
            time.sleep(backoff_sec * attempt)
    return None

def sma(series: pd.Series, length: int) -> pd.Series:
    return series.rolling(length).mean()

def two_red(df: pd.DataFrame) -> pd.Series:
    red_prev = df['Close'].shift(1) < df['Open'].shift(1)
    red_now  = df['Close'] < df['Open']
    return (red_prev & red_now)

def build_base_signals(df: pd.DataFrame) -> pd.DataFrame:
    df['DMA_fast'] = sma(df['Close'], DMA_FAST)
    df['DMA_slow'] = sma(df['Close'], DMA_SLOW)
    df['two_red'] = two_red(df)
    return df

def get_stock_data(ticker: str, start: str, end: str) -> pd.DataFrame:
    raw = _read_cache(ticker)
    if raw is None:
        raw = download_with_retry(ticker, start, end, interval="1d", max_retries=3, backoff_sec=2)
        if raw is None or raw.empty:
            return None
        raw = normalize_df_columns(raw)
        raw = collapse_duplicate_columns_take_first(raw)
        _write_cache(ticker, raw)

    df = raw.copy()
    df = normalize_df_columns(df)
    df = collapse_duplicate_columns_take_first(df)

    for c in ['Close','High','Low','Open','Volume']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    df = build_base_signals(df)
    df.dropna(inplace=True)
    return df if not df.empty else None

def send_telegram(text: str) -> None:
    if not ENABLE_TELEGRAM:
        print("[DRY] Telegram disabled. Message would be:\n", text)
        return
    token = TELEGRAM_BOT_TOKEN
    chat_id = TELEGRAM_CHAT_ID
    if not token or not chat_id:
        print("Telegram ENV missing; skipping send.")
        print(text)
        return
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    try:
        requests.post(url, data={"chat_id": chat_id, "text": text, "parse_mode": "Markdown"})
    except Exception as e:
        print("Telegram send error:", e)
        print(text)

# ----------- Exit logic -----------
def check_exit_for_position(row_pos: pd.Series, df: pd.DataFrame) -> Dict:
    """
    row_pos: one row from open_positions.csv
    df: full daily df for ticker with DMA and two_red
    Returns dict with 'should_exit', 'reason', 'trigger_price', 'as_of_date', 'days_held', 'peak_price'
    """
    ticker = row_pos['ticker']
    entry_date = pd.to_datetime(row_pos['entry_date']).date()
    entry_price = float(row_pos['entry_price'])

    protective_exit = bool(row_pos.get('protective_exit', True))
    use_hsl = bool(row_pos.get('use_hard_stop', False))
    hsl_pct = float(row_pos.get('hard_stop_pct', 5.0))
    use_tsl = bool(row_pos.get('use_trailing_stop', False))
    tsl_pct = float(row_pos.get('trailing_stop_pct', 10.0))
    max_hold_days = int(row_pos.get('max_hold_days', 10))
    prev_peak = float(row_pos.get('peak_price', entry_price))

    if df is None or df.empty:
        return {"should_exit": False}

    # restrict to bars since entry
    df_since = df[df.index.date >= entry_date]
    if df_since.empty:
        return {"should_exit": False}

    # compute peak high since entry
    peak_price = max(prev_peak, float(df_since['High'].max()))

    # evaluate on the latest bar
    cur = df.iloc[-1]
    as_of_date = cur.name.date()
    days_held = (df_since.shape[0] - 1)  # bars since entry, excluding entry bar

    # thresholds
    hard_stop_price = entry_price * (1.0 - hsl_pct/100.0) if use_hsl else None
    trailing_stop_price = peak_price * (1.0 - tsl_pct/100.0) if use_tsl else None

    # priority: HSL -> TSL -> Time -> TwoRedBelow20 -> Protective
    # 1) HSL
    if use_hsl and (cur['Low'] <= hard_stop_price):
        return {
            "should_exit": True, "reason": "HSL",
            "trigger_price": hard_stop_price, "as_of_date": as_of_date,
            "days_held": days_held, "peak_price": peak_price
        }

    # 2) TSL
    if use_tsl and (cur['Low'] <= trailing_stop_price):
        return {
            "should_exit": True, "reason": "TSL",
            "trigger_price": trailing_stop_price, "as_of_date": as_of_date,
            "days_held": days_held, "peak_price": peak_price
        }

    # 3) Time
    if days_held >= max_hold_days:
        return {
            "should_exit": True, "reason": "Time",
            "trigger_price": float(cur['Close']), "as_of_date": as_of_date,
            "days_held": days_held, "peak_price": peak_price
        }

    # 4) TwoRedBelow20
    if bool(cur['two_red'] and (cur['Close'] < cur['DMA_slow'])):
        return {
            "should_exit": True, "reason": "TwoRedBelow20",
            "trigger_price": float(cur['Close']), "as_of_date": as_of_date,
            "days_held": days_held, "peak_price": peak_price
        }

    # 5) Protective
    if protective_exit and (cur['Close'] < cur['DMA_slow']):
        return {
            "should_exit": True, "reason": "Protective",
            "trigger_price": float(cur['Close']), "as_of_date": as_of_date,
            "days_held": days_held, "peak_price": peak_price
        }

    # no exit
    return {
        "should_exit": False, "reason": None,
        "as_of_date": as_of_date, "days_held": days_held, "peak_price": peak_price
    }

def main():
    end = END_DATE
    if end is None:
        end = (datetime.utcnow() + timedelta(days=1)).strftime("%Y-%m-%d")

    if not OPEN_POSITIONS_CSV.exists():
        print("No open positions file found:", OPEN_POSITIONS_CSV)
        return

    open_df = pd.read_csv(OPEN_POSITIONS_CSV, parse_dates=['entry_date'])
    if open_df.empty:
        print("No open positions to check.")
        return

    # group by ticker to minimize downloads
    tickers = sorted(open_df['ticker'].unique().tolist())
    data_by_ticker = {}
    for t in tickers:
        try:
            df = get_stock_data(t, START_DATE, end)
        except Exception as e:
            print(f"[{t}] error: {e}\n{traceback.format_exc()}")
            df = None
        data_by_ticker[t] = df

    alerts = []
    keep_rows = []
    for _, pos in open_df.iterrows():
        t = pos['ticker']
        df = data_by_ticker.get(t)
        res = check_exit_for_position(pos, df)

        # always update peak & days_held in the stored file
        pos['peak_price'] = res.get('peak_price', pos.get('peak_price', pos['entry_price']))
        pos['days_held_calc'] = res.get('days_held', 0)

        if res.get('should_exit', False):
            alerts.append((t, res['reason'], res['as_of_date'], float(res['trigger_price'])))
            if not AUTO_REMOVE_ON_ALERT:
                keep_rows.append(pos)  # keep but mark? You can add a status field if you prefer.
        else:
            keep_rows.append(pos)

    # Save updated open positions (with peak & days_held_calc)
    pd.DataFrame(keep_rows).to_csv(OPEN_POSITIONS_CSV, index=False)

    # Send Telegram grouped by date
    if alerts:
        # write per-date CSV and message
        per_date = {}
        for (t, reason, d, price) in alerts:
            per_date.setdefault(d, []).append((t, reason, price))

        for d, items in sorted(per_date.items()):
            out = pd.DataFrame([{"ticker": t, "reason": r, "trigger_price": price} for (t, r, price) in items])
            outpath = EXIT_ALERTS_DIR / f"exits_{d}.csv"
            out.to_csv(outpath, index=False)

        # single message summary
        lines = ["*Exit Alerts — DMA(10,20)+2-Green*"]
        for d, items in sorted(per_date.items()):
            tick_str = ", ".join([f"{t}({r})" for (t, r, _) in items])
            lines.append(f"*{d}* — {tick_str}")
        send_telegram("\n".join(lines))

        print("Alerts sent.")
    else:
        print("No exits triggered today.")

if __name__ == "__main__":
    main()


No exits triggered today.


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_4788/3293788840.py:251: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = (datetime.utcnow() + timedelta(days=1)).strftime("%Y-%m-%d")
